In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv  # Using Chebyshev convolution
from torch.utils.data import Dataset
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import knn_graph


In [7]:
class DiffusionNetLayer(torch.nn.Module):
    def __init__(self, in_features, out_features, K=6):
        super(DiffusionNetLayer, self).__init__()
        # Chebyshev convolution approximates diffusion on graphs
        self.conv = ChebConv(in_features, out_features, K)

    def forward(self, x, edge_index, edge_weight=None):
        # Perform the convolution on the graph using edge weights (sparse Laplacian)
        x = self.conv(x, edge_index, edge_weight=edge_weight)
        return F.relu(x)
    
class Encoder(torch.nn.Module):
    def __init__(self, in_features, hidden_features, latent_dim):
        super(Encoder, self).__init__()
        self.diffusion1 = DiffusionNetLayer(in_features, hidden_features)
        self.diffusion2 = DiffusionNetLayer(hidden_features, latent_dim)

    def forward(self, x, edge_index, edge_weight):
        x = self.diffusion1(x, edge_index, edge_weight)
        x = self.diffusion2(x, edge_index, edge_weight)
        return x

class Decoder(torch.nn.Module):
    def __init__(self, latent_dim, hidden_features, out_features):
        super(Decoder, self).__init__()
        self.diffusion1 = DiffusionNetLayer(latent_dim, hidden_features)
        self.diffusion2 = DiffusionNetLayer(hidden_features, out_features)

    def forward(self, x, edge_index, edge_weight):
        x = self.diffusion1(x, edge_index, edge_weight)
        x = self.diffusion2(x, edge_index, edge_weight)
        return x

class DiffusionNetAutoencoder(torch.nn.Module):
    def __init__(self, in_features, hidden_features, latent_dim):
        super(DiffusionNetAutoencoder, self).__init__()
        self.encoder = Encoder(in_features, hidden_features, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_features, in_features)

    def forward(self, x, edge_index, edge_weight):
        # Encode to latent space
        latent = self.encoder(x, edge_index, edge_weight)
        # Decode back to original space
        reconstructed = self.decoder(latent, edge_index, edge_weight)
        return reconstructed

In [8]:
class PointCloudGraphDataset(Dataset):
    def __init__(self, point_clouds, k=6):
        """
        Args:
            point_clouds (Tensor): Tensor of shape [num_clouds, num_points, dim_point].
            k (int): Number of nearest neighbors for constructing edge index.
        """
        self.point_clouds = point_clouds  # shape: [num_clouds, num_points, dim_point]
        self.k = k  # k-nearest neighbors

    def __len__(self):
        return self.point_clouds.shape[0]

    def __getitem__(self, idx):
        # Get the point cloud at index idx
        point_cloud = self.point_clouds[idx]  # shape: [num_points, dim_point]

        # Create edge_index using k-nearest neighbors (kNN)
        edge_index = knn_graph(point_cloud, k=self.k, loop=False)  # shape: [2, num_edges]

        # Prepare data for PyTorch Geometric Data object
        data = Data(x=point_cloud, edge_index=edge_index)

        return data

# Example dataset: Random point clouds
num_clouds = 1000  # Number of point clouds
num_points = 100   # Number of points per cloud
dim_point = 3      # Dimensionality of each point (e.g., 3D points)

# Simulate a dataset of random point clouds
point_clouds = torch.rand((num_clouds, num_points, dim_point))

# Create dataset and dataloader
k = 6  # Number of nearest neighbors for graph construction
dataset = PointCloudGraphDataset(point_clouds, k=k)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example DiffusionNetLayer
model = DiffusionNetLayer(in_features=dim_point, out_features=16, K=6)

# Training loop example
for batch in []:#dataloader:
    x = batch.x  # Node features for all graphs in the batch
    edge_index = batch.edge_index  # Edge indices for all graphs in the batch
    
    # Forward pass through the model
    output = model(x, edge_index)
    print (f"output{output}")
    
    # Compute loss, backpropagation, etc.


In [ ]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import knn_graph
from torch_geometric.utils import degree

class PointCloudGraphDataset(Dataset):
    def __init__(self, point_clouds, k=6):
        """
        Args:
            point_clouds (Tensor): Tensor of shape [num_clouds, num_points, dim_point].
            k (int): Number of nearest neighbors for constructing edge index.
        """
        self.point_clouds = point_clouds  # shape: [num_clouds, num_points, dim_point]
        self.k = k  # k-nearest neighbors

    def __len__(self):
        return self.point_clouds.shape[0]

    def __getitem__(self, idx):
        # Get the point cloud at index idx
        point_cloud = self.point_clouds[idx]  # shape: [num_points, dim_point]

        # Create edge_index using k-nearest neighbors (kNN)
        edge_index = knn_graph(point_cloud, k=self.k, loop=False)  # shape: [2, num_edges]

        # Compute Laplacian (sparse form using edge weights)
        edge_weight = self.compute_edge_weight(edge_index, point_cloud.size(0))

        # Prepare data for PyTorch Geometric Data object
        data = Data(x=point_cloud, edge_index=edge_index, edge_weight=edge_weight)

        return data

    def compute_edge_weight(self, edge_index, num_nodes):
        """
        Compute edge weights corresponding to the Laplacian matrix in sparse form.
        
        Args:
            edge_index (Tensor): Edge indices for the graph.
            num_nodes (int): Number of nodes (points) in the graph.
        
        Returns:
            edge_weight (Tensor): Edge weights corresponding to the Laplacian.
        """
        # Compute degree for each node
        row, col = edge_index
        deg = degree(row, num_nodes=num_nodes, dtype=torch.float)

        # Compute edge weights for Laplacian: L = D - A
        edge_weight = torch.ones(edge_index.size(1), dtype=torch.float)  # Adjacency weights (1 for each edge)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        # Apply normalized Laplacian: D^{-0.5} A D^{-0.5}
        edge_weight = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

        return edge_weight


# Example dataset: Random point clouds
num_clouds = 1000  # Number of point clouds
num_points = 100   # Number of points per cloud
dim_point = 3      # Dimensionality of each point (e.g., 3D points)

# Simulate a dataset of random point clouds
point_clouds = torch.rand((num_clouds, num_points, dim_point))

# Create dataset and dataloader with Laplacian computation
k = 6  # Number of nearest neighbors for graph construction
dataset = PointCloudGraphDataset(point_clouds, k=k)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example DiffusionNetLayer
# model = DiffusionNetLayer(in_features=dim_point, out_features=16, K=6)
hidden_features = 64
latent_dim = 32
model = DiffusionNetAutoencoder(dim_point, hidden_features, latent_dim)

# Training loop example
for batch in dataloader:
    x = batch.x  # Node features for all graphs in the batch
    edge_index = batch.edge_index  # Edge indices for all graphs in the batch
    edge_weight = batch.edge_weight  # Edge weights (Laplacian) for all graphs in the batch
    
    # Forward pass through the model
    output = model(x, edge_index, edge_weight=edge_weight)  # Use edge_weight as laplacian
    print (f"output{output}")
    # Compute loss, backpropagation, etc.
